In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import random 
import stanza
from stanza.server import CoreNLPClient
import emoji
import spacy
from spacy.matcher import Matcher
import random
from textblob import Word
import re
from nltk.tokenize import word_tokenize, sent_tokenize


/Users/henry/miniforge3/envs/mlenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Original data labelled

In [2]:
df = pd.read_csv("data/labelled_data.csv").drop(["Unnamed: 0"], axis = 1)
df.head()

,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,n_photo_user,url_user,restraunt_name,food_sentiment,service_sentiment,ambience_sentiment,other_sentiment
0,ChdDSUhNMG9nS0VJQ0FnSURoN2IzU2hRRRAB,Took my son there for dinner and we had a good...,5 days ago,28:14.0,5,laurent,0,0,https://www.google.com/maps/contrib/1052889272...,Hooters,1,1,0,1
1,ChZDSUhNMG9nS0VJQ0FnSURodHVDeWRREAE,Pathetic food. Ordered vegetarian burger. Tast...,a week ago,28:14.0,1,Fa Ris,0,0,https://www.google.com/maps/contrib/1061823533...,Hooters,-1,0,0,-1
2,ChZDSUhNMG9nS0VJQ0FnSURoLW9HUEhREAE,"Friendly staff, superb service, food was serve...",a week ago,28:14.0,5,Florencio Gali,0,0,https://www.google.com/maps/contrib/1074077044...,Hooters,1,1,0,0
3,ChZDSUhNMG9nS0VJQ0FnSURoaXVyTkhREAE,I have yet to find a Hooter's that I didn't li...,2 weeks ago,28:14.0,5,Joshua Manning,0,0,https://www.google.com/maps/contrib/1083143598...,Hooters,1,0,0,1
4,ChZDSUhNMG9nS0VJQ0FnSURock9MV1pREAE,"Alcohol is expensive everywhere in Singapore, ...",2 weeks ago,28:14.0,4,J.K LEE,0,0,https://www.google.com/maps/contrib/1101703195...,Hooters,1,1,0,1


* data1 : this contains aspects phrases/sentences extracted using method 1 
    * code for extraction in Extracting_Aspects_1.ipynb (cannot be runned without api key)

In [3]:
data1 = pd.read_csv("data/extracted_aspects_raw.csv").drop(["Unnamed: 0"], axis = 1)
data1

,food_aspect,service_aspect,ambience_aspect
0,Philly cheese steak was really good,"Service was good, girls are very pleasant and ...",NaN
1,"Pathetic food, Ordered vegetarian burger, tast...",NaN,NaN
2,"food was serve hot and fresh, not so pricey,","Friendly staff, superb service, pretty service...",Place is a bit outdated might needs some upgra...
3,"Cold beer, hot wings",NaN,NaN
4,The chicken is delicious,"There was a beer 2+1 promotion, it was nice to...",NaN
...,...,...,...
3887,The food was really good,"the service was excellent, amb: th",the ambiance was cozy and inviting.
3888,'When the food came they refused to credit us ...,'Our order taker was a bit blur and accidental...,'I come here a lot and used to think that the ...
3889,"a bit so-so, didn't try the specialty coffee",NaN,NaN
3890,"French toast was nice but too sweet, coffee wa...",NaN,NaN


## For each document in an aspect 
- Remove punctuations
- Remove stopwords
- Lemmatize
- convert emojis into text
- NaN just means there are no parts of the original comments that describe that particular aspect, we shud keep it (replace it with ""). we feel that a no mention shud default to a "neutral" sentiment

In [4]:
# some "NA" string in the data1 convert it to empty string
data1.loc[data1.food_aspect.isin(["NA"," NA", "NA "]), "food_aspect"] = ""
data1.loc[data1.service_aspect.isin(["NA"," NA", "NA "]), "service_aspect"] = ""
data1.loc[data1.ambience_aspect.isin(["NA"," NA", "NA "]), "ambience_aspect"] = ""



#replace all NA's with empty string
data1.loc[data1.food_aspect.isna(), "food_aspect"] = ""
data1.loc[data1.service_aspect.isna(), "service_aspect"] = ""
data1.loc[data1.ambience_aspect.isna(), "ambience_aspect"] = ""
data1

,food_aspect,service_aspect,ambience_aspect
0,Philly cheese steak was really good,"Service was good, girls are very pleasant and ...",
1,"Pathetic food, Ordered vegetarian burger, tast...",,
2,"food was serve hot and fresh, not so pricey,","Friendly staff, superb service, pretty service...",Place is a bit outdated might needs some upgra...
3,"Cold beer, hot wings",,
4,The chicken is delicious,"There was a beer 2+1 promotion, it was nice to...",
...,...,...,...
3887,The food was really good,"the service was excellent, amb: th",the ambiance was cozy and inviting.
3888,'When the food came they refused to credit us ...,'Our order taker was a bit blur and accidental...,'I come here a lot and used to think that the ...
3889,"a bit so-so, didn't try the specialty coffee",,
3890,"French toast was nice but too sweet, coffee wa...",,


In [5]:
data1.iloc[280:290] # contains emojis

,food_aspect,service_aspect,ambience_aspect
280,Very good,as always,
281,Love the food. Yummyy,,
282,price for food and drinks didn't meet our expe...,,"attractive from outside, experience at any oth..."
283,very good,very friendly,
284,,,Nice place to chill out
285,Poor gluten free choices,,
286,1 for 1 wings,service is pretty bad,
287,"good quality, reasonably priced,",,
288,"Ordered daltona sauce chicken wings, Loved it👍",,
289,The food and drinks are not too expensive,,The location is the best thing there and bang ...


In [6]:
def replace_emojis(text):
    all_emojis = emoji.get_aliases_unicode_dict().values()
    for character in text:
        if character in all_emojis:
            emote_in_text = emoji.demojize(character).replace(":"," ")
            text = text.replace(character, emote_in_text)
    return text

data1["food_aspect"] = data1.food_aspect.apply(lambda x : replace_emojis(x))
data1["service_aspect"] = data1.service_aspect.apply(lambda x : replace_emojis(x))
data1["ambience_aspect"] = data1.ambience_aspect.apply(lambda x : replace_emojis(x))

In [7]:
data1.food_aspect[288] # check if emoji is replaced (it is)

'Ordered daltona sauce chicken wings, Loved it thumbs_up '

In [8]:
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
punctuations = string.punctuation


def process_doc(corp, stop_words):
    with CoreNLPClient(annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'], memory='8G', endpoint='http://localhost:9001', be_quiet=True) as client:
        # collect all lemmas
        lemmas = []
        for doc in corp:
            #print(doc)
            lemma = []
            document = client.annotate(doc)
            for sent in document.sentence:
                for token in sent.token:
                    if token.lemma not in punctuations and token.lemma not in stop_words: # drop all punctuations and remove all stopwords
                        lemma.append(token.lemma.lower()) # lower case everything
            lemmas.append(" ".join(lemma))
    return lemmas

[nltk_data] Downloading package stopwords to /Users/henry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/henry/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Processing the data using aspect extraction 1
-  The extraction is already done in Extracting_Aspects_1.ipynb
- remove stopwords, remove puncuations, lemmtize, replace emojis with their word representation

In [9]:
food_aspect_extraction_1 = process_doc(data1.food_aspect, stop_words= stop_words)
service_aspect_extraction_1 = process_doc(data1.service_aspect, stop_words= stop_words)
ambience_aspect_extraction_1 = process_doc(data1.ambience_aspect, stop_words= stop_words)

2023-04-03 11:24:11 INFO: Writing properties to tmp file: corenlp_server-4e04ac7508384fe6.props
2023-04-03 11:24:11 INFO: Starting server with command: java -Xmx8G -cp /Users/henry/stanza_corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-4e04ac7508384fe6.props -annotators tokenize,ssplit,pos,lemma,ner,parse,depparse,coref -preload -outputFormat serialized
2023-04-03 11:27:28 INFO: Writing properties to tmp file: corenlp_server-ea0ad9caef684eef.props
2023-04-03 11:27:28 INFO: Starting server with command: java -Xmx8G -cp /Users/henry/stanza_corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-ea0ad9caef684eef.props -annotators tokenize,ssplit,pos,lemma,ner,parse,depparse,coref -preload -outputFormat serialized
2023-04-03 11:29:14 INFO: Writing properties to tmp file: co

In [10]:
aspect_extraction_1 = pd.DataFrame({"food_aspect_extraction_1": food_aspect_extraction_1, "service_aspect_extraction_1":service_aspect_extraction_1,  "ambience_aspect_extraction_1" : ambience_aspect_extraction_1})
aspect_extraction_1

,food_aspect_extraction_1,service_aspect_extraction_1,ambience_aspect_extraction_1
0,philly cheese steak really good,service good girl pleasant pay attention servi...,
1,pathetic food order vegetarian burger taste aw...,,
2,food serve hot fresh pricey,friendly staff superb service pretty service s...,place bit outdated might need upgrading
3,cold beer hot wing,,
4,chicken delicious,beer 2 +1 promotion nice take picture staff,
...,...,...,...
3887,food really good,service excellent amb th,ambiance cozy invite
3888,food come refuse credit additional main,order taker bit blur accidentally charge four ...,i come lot use think hipster vibe mean place c...
3889,bit try specialty coffee,,
3890,french toast nice sweet coffee good cake,,


# Aspect extraction 2 (YH method)
* Give some explanation here

In [11]:
FOOD_LEXICONS = set(pd.read_csv("data/yelp/food.csv", header=None, names = ["food"])["food"].str.lower())
AMBIENCE_LEXICONS = set(pd.read_csv("data/yelp/ambience.csv", header=None, names = ["ambience"])["ambience"].str.lower())
SERVICE_LEXICONS = set(pd.read_csv("data/yelp/service.csv", header=None, names = ["service"])["service"].str.lower())

In [12]:
#addition of lexicons
new_food = set(['food','dish'])
new_ser = set(['staff','waiter','waitress','manager','person','lady','man',
          'cashier','bartender','barista','owner'])
new_amb = set(['environment','lighting','noise','decor','view','music','place',
          'furniture'])

FOOD_LEXICONS.update(new_food)
SERVICE_LEXICONS.update(new_ser)
AMBIENCE_LEXICONS.update(new_amb)

In [13]:
aspects = ["food","ambience","service"]
ASPECTS_LEXICONS = [FOOD_LEXICONS,AMBIENCE_LEXICONS,SERVICE_LEXICONS]

In [14]:
flat_list = [item for sublist in ASPECTS_LEXICONS for item in sublist]

In [15]:
nlp = spacy.load("en_core_web_sm")


def match_pattern(text):
    matcher = Matcher(nlp.vocab)
    patterns = [
            [{"POS": "ADV", "OP": "{,1}"},{"POS": "ADJ"},{"POS" : "NOUN"}], #for descriptive word -> noun (w adv)
            [{"POS" : "NOUN"}, {"POS": "ADV", "OP" : "{,1}"},{"POS": "ADJ"}],
            [{"POS": "ADV", "OP": "{,1}"},{"POS": "ADJ"},{"POS" : "PROPN"}],
            [{"POS" : "PROPN"}, {"POS": "ADV", "OP" : "{,1}"},{"POS": "ADJ"}],
            [{"POS" : "ADV", "OP" : "{,1}"},{"POS": "ADJ"}],
            [{"POS" : "NOUN"}, {"POS" : "VERB"}, {"POS": "ADV", "OP" : "{,1}"},{"POS": "ADJ"}],
            [{"POS" : "NOUN"}, {"POS": "AUX", "OP" : "{,1}"},{"POS" : "VERB", "OP" :"{,1}"},{"POS": "ADJ"}]
          ]
    #but got repeats when the adv comes in...
    matcher.add("test", patterns)
    
    doc = nlp(text)
    matches = matcher(doc)
    chunk_list = []
    for match_id, start,end in matches:
        span = doc[start:end]  # The matched span
        chunk_list.append([span.text])
    return chunk_list

In [16]:
def food_senti(senti_list): #input is a list of list of chunks
    output = []
    for chunk in senti_list:
        chunk = chunk[0]
        doc = nlp(chunk)
        if any(token.text in FOOD_LEXICONS for token in doc):
            output.append(chunk)
    if output:
        return output
    else:
        return np.nan
    
def service_senti(senti_list): #input is a list of list containing str or token?
    output = []
    for chunk in senti_list:
        chunk = chunk[0]
        doc = nlp(chunk)
        if any(token.text in SERVICE_LEXICONS for token in doc):
            output.append(chunk)
    if output:
        return output
    else:
        return np.nan
    
def ambience_senti(senti_list): 
    output = []
    for chunk in senti_list:
        chunk = chunk[0]
        doc = nlp(chunk)
        if any(token.text in AMBIENCE_LEXICONS for token in doc):
            output.append(chunk)
    if output:
        return output
    else:
        return np.nan

In [17]:
### custom stop words
#create new stop list that excludes adv and adv and CCONJ 
#can use this new stoplist to preprocess

from nltk.corpus import stopwords

custom_stop = stopwords.words('english')
for word in custom_stop:
    for token in nlp(word):
        if token.pos_ == "ADV" or token.pos_ == "ADJ" or token.pos_ == "CCONJ":
            custom_stop.remove(token.text)

In [18]:
aspect_extraction_2 = pd.DataFrame({"processed_reviews": process_doc(df.caption,custom_stop)})
aspect_extraction_2

2023-04-03 11:30:38 INFO: Writing properties to tmp file: corenlp_server-a457a9d9bda74f32.props
2023-04-03 11:30:38 INFO: Starting server with command: java -Xmx8G -cp /Users/henry/stanza_corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-a457a9d9bda74f32.props -annotators tokenize,ssplit,pos,lemma,ner,parse,depparse,coref -preload -outputFormat serialized


,processed_reviews
0,take son dinner and good time eat philly chees...
1,pathetic food order vegetarian burger taste aw...
2,friendly staff superb service food serve hot a...
3,i yet find hooter 's i like cold beer hot wing
4,alcohol expensive everywhere singapore i go ho...
...,...
3887,favorite muddy espresso
3888,order taker bit blur and accidentally charge f...
3889,go lunch bit try specialty coffee rating might...
3890,hit and miss french toast nice too sweet !! co...


In [19]:
aspect_extraction_2["match_pattern"] = aspect_extraction_2.processed_reviews.apply(lambda x : match_pattern(x))
aspect_extraction_2

,processed_reviews,match_pattern
0,take son dinner and good time eat philly chees...,"[[good], [good time], [really good], [good], [..."
1,pathetic food order vegetarian burger taste aw...,"[[pathetic], [pathetic food], [burger taste aw..."
2,friendly staff superb service food serve hot a...,"[[friendly], [friendly staff], [food serve hot..."
3,i yet find hooter 's i like cold beer hot wing,"[[cold], [cold beer], [beer hot], [hot], [hot ..."
4,alcohol expensive everywhere singapore i go ho...,"[[alcohol expensive], [expensive], [+1], [+1 p..."
...,...,...
3887,favorite muddy espresso,"[[favorite], [muddy], [muddy espresso]]"
3888,order taker bit blur and accidentally charge f...,"[[main], [main course], [credit additional], [..."
3889,go lunch bit try specialty coffee rating might...,"[[rating might bit biased], [biased]]"
3890,hit and miss french toast nice too sweet !! co...,"[[french], [french toast], [toast nice], [nice..."


In [20]:
aspect_extraction_2["food_aspect_extraction_2"] = aspect_extraction_2.match_pattern.apply(lambda x : food_senti(x))
aspect_extraction_2["service_aspect_extraction_2"] = aspect_extraction_2.match_pattern.apply(lambda x: service_senti(x))
aspect_extraction_2["ambience_aspect_extraction_2"] = aspect_extraction_2.match_pattern.apply(lambda x: ambience_senti(x))
aspect_extraction_2

,processed_reviews,match_pattern,food_aspect_extraction_2,service_aspect_extraction_2,ambience_aspect_extraction_2
0,take son dinner and good time eat philly chees...,"[[good], [good time], [really good], [good], [...",NaN,"[really good service, good service, service good]",NaN
1,pathetic food order vegetarian burger taste aw...,"[[pathetic], [pathetic food], [burger taste aw...","[pathetic food, burger taste awful, awful pota...",NaN,NaN
2,friendly staff superb service food serve hot a...,"[[friendly], [friendly staff], [food serve hot...",[food serve hot],"[friendly staff, pretty service]",NaN
3,i yet find hooter 's i like cold beer hot wing,"[[cold], [cold beer], [beer hot], [hot], [hot ...","[cold beer, beer hot]",NaN,NaN
4,alcohol expensive everywhere singapore i go ho...,"[[alcohol expensive], [expensive], [+1], [+1 p...",NaN,NaN,NaN
...,...,...,...,...,...
3887,favorite muddy espresso,"[[favorite], [muddy], [muddy espresso]]",[muddy espresso],NaN,NaN
3888,order taker bit blur and accidentally charge f...,"[[main], [main course], [credit additional], [...",NaN,NaN,NaN
3889,go lunch bit try specialty coffee rating might...,"[[rating might bit biased], [biased]]",NaN,NaN,NaN
3890,hit and miss french toast nice too sweet !! co...,"[[french], [french toast], [toast nice], [nice...","[french toast, toast nice]",NaN,NaN


In [21]:
aspect_extraction_2["food_aspect_extraction_2"] = aspect_extraction_2.food_aspect_extraction_2.apply(lambda x : "" if x is np.nan else " ".join(x))
aspect_extraction_2["service_aspect_extraction_2"] = aspect_extraction_2.service_aspect_extraction_2.apply(lambda x: "" if x is np.nan else " ".join(x))
aspect_extraction_2["ambience_aspect_extraction_2"] = aspect_extraction_2.ambience_aspect_extraction_2.apply(lambda x: "" if x is np.nan else " ".join(x))


In [22]:
aspect_extraction_2 = aspect_extraction_2.drop(["processed_reviews","match_pattern"],axis = 1)
aspect_extraction_2

,food_aspect_extraction_2,service_aspect_extraction_2,ambience_aspect_extraction_2
0,,really good service good service service good,
1,pathetic food burger taste awful awful potato ...,,
2,food serve hot,friendly staff pretty service,
3,cold beer beer hot,,
4,,,
...,...,...,...
3887,muddy espresso,,
3888,,,
3889,,,
3890,french toast toast nice,,


In [23]:
# print("Example review:")
# example = df.caption[2]
# # print the text out nicely,seperate by sentences
# x = " "
# for i in example:
#     if i != ".":
#         x += i
#     else:
#         print(x)
#         x = ""
# print("------------------")
# aspect_extraction_2.iloc[2,:]

# Aspect extraction 3 (CY method)
* some explanation

In [24]:
aspect_extraction_3 = df[["caption"]]
aspect_extraction_3.caption = aspect_extraction_3.caption.apply(lambda x : replace_emojis(x))
aspect_extraction_3.caption = aspect_extraction_3.caption.apply(lambda x : re.sub(r'[^A-Za-z0-9 ]+', '', x ))
aspect_extraction_3

/var/folders/vg/q5w3xdtj6cq6f0pfq2dpdgkh0000gn/T/ipykernel_35024/3968348992.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aspect_extraction_3.caption = aspect_extraction_3.caption.apply(lambda x : replace_emojis(x))
/var/folders/vg/q5w3xdtj6cq6f0pfq2dpdgkh0000gn/T/ipykernel_35024/3968348992.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aspect_extraction_3.caption = aspect_extraction_3.caption.apply(lambda x : re.sub(r'[^A-Za-z0-9 ]+', '', x ))


,caption
0,Took my son there for dinner and we had a good...
1,Pathetic food Ordered vegetarian burger Tasted...
2,Friendly staff superb service food was serve h...
3,I have yet to find a Hooters that I didnt like...
4,Alcohol is expensive everywhere in Singapore s...
...,...
3887,My favorite is muddy espresso
3888,Our order taker was a bit blur and accidentall...
3889,Went for lunch a bit soso didnt try the specia...
3890,Some hits and some misses The french toast was...


In [25]:
foodLex = list(pd.read_csv("data/yelp/food.csv", header=None, names = ["food"])["food"].str.lower())
# some extra food lexicons
foodLex.append(['food','dish'])
serLex = list(pd.read_csv("data/yelp/service.csv", header=None, names = ["service"])["service"].str.lower())
# some extra service lexicons
serLex.append(['staff', 'waiter','waitress','manager','person','lady','man','cashier' 'bartender' ,'chef','owner','barista'])
ambLex = list(pd.read_csv("data/yelp/ambience.csv", header=None, names = ["ambience"])["ambience"].str.lower())
ambLex.append(['environment','lighting','noise','decor','view','music','place'])

lex = [foodLex, serLex, ambLex]

In [26]:
def extract_text(review, aspect):
    
    sentences = sent_tokenize(review.lower())
    
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for sentence in sentences for word in word_tokenize(sentence) if word.isalnum()]
    
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

        
    keyword_list = aspect
    relevant_text = ''
    for i, word in enumerate(words):
        if word in keyword_list:
            relevant_text = ''
            j = i + 1
            relevant_words = words[max(i-5, 0):min(j+10, len(words))]
            wordindex = relevant_words.index(word)
            for k in range(len(relevant_words)):
                wordtype = nltk.pos_tag([relevant_words[k]])[0][1]
                if wordtype in ['JJ','JJR','JJS','RB','RBR','RBS']:
                    if k < wordindex:
                        keywords = relevant_words[k:wordindex+1]
                        relevant_text = ' '.join(keywords)
                        break
                    else:
                        if wordtype in ['RB','RBR','RBS']:
                            keywords = relevant_words[wordindex:k+2]
                            relevant_text = ' '.join(keywords)
                        else:
                            keywords = relevant_words[wordindex:k+1]
                            relevant_text = ' '.join(keywords)
                        break


            
    return relevant_text

In [27]:
aspect_extraction_3["food_aspect_extraction_3"] = aspect_extraction_3.caption.apply(lambda x : extract_text(x,lex[0])) # lex[0] == food lexicons
aspect_extraction_3["service_aspect_extraction_3"] = aspect_extraction_3.caption.apply(lambda x : extract_text(x,lex[1])) # lex[0] == food lexicons
aspect_extraction_3["ambience_aspect_extraction_3"] = aspect_extraction_3.caption.apply(lambda x : extract_text(x,lex[2])) # lex[0] == food lexicons

/var/folders/vg/q5w3xdtj6cq6f0pfq2dpdgkh0000gn/T/ipykernel_35024/2654852495.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aspect_extraction_3["food_aspect_extraction_3"] = aspect_extraction_3.caption.apply(lambda x : extract_text(x,lex[0])) # lex[0] == food lexicons
/var/folders/vg/q5w3xdtj6cq6f0pfq2dpdgkh0000gn/T/ipykernel_35024/2654852495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aspect_extraction_3["service_aspect_extraction_3"] = aspect_extraction_3.caption.apply(lambda x : extract_text(

In [28]:
aspect_extraction_3

,caption,food_aspect_extraction_3,service_aspect_extraction_3,ambience_aspect_extraction_3
0,Took my son there for dinner and we had a good...,good time ate philly cheese steak,good girl pleasant pay attention service,
1,Pathetic food Ordered vegetarian burger Tasted...,good also beware happy hour beer,,
2,Friendly staff superb service food was serve h...,,hot fresh pricey pretty service,
3,I have yet to find a Hooters that I didnt like...,beer hot,,
4,Alcohol is expensive everywhere in Singapore s...,delicious thing see wa beer,,
...,...,...,...,...
3887,My favorite is muddy espresso,,,
3888,Our order taker was a bit blur and accidentall...,,,
3889,Went for lunch a bit soso didnt try the specia...,,,
3890,Some hits and some misses The french toast was...,nice sweet coffee wa good cake,,


In [37]:
# print("Example review:")
# example = df.caption[0]
# # print the text out nicely,seperate by sentences
# x = " "
# for i in example:
#     if i != ".":
#         x += i
#     else:
#         print(x)
#         x = ""
# print("------------------")
# aspect_extraction_3.iloc[0,:]

Example review:
 Took my son there for dinner and we had a good time
 We both ate a Philly cheese steak and it was really good
 Service was good, girls are very pleasant and pay attention to the service, prices are very reasonable
------------------


food_aspect_extraction_3               good time ate philly cheese steak
service_aspect_extraction_3     good girl pleasant pay attention service
ambience_aspect_extraction_3                                            
Name: 0, dtype: object

In [29]:
# change all list values into text
# aspect_extraction_3["food_aspect_extraction_3"] = aspect_extraction_3.food_aspect_extraction_3.apply(lambda x : "" if x is np.nan else " ".join(x))
# aspect_extraction_3["service_aspect_extraction_3"] = aspect_extraction_3.service_aspect_extraction_3.apply(lambda x: "" if x is np.nan else " ".join(x))
# aspect_extraction_3["ambience_aspect_extraction_3"] = aspect_extraction_3.ambience_aspect_extraction_3.apply(lambda x: "" if x is np.nan else " ".join(x))

In [30]:
aspect_extraction_3 = aspect_extraction_3.drop("caption", axis=1)
aspect_extraction_3 

,food_aspect_extraction_3,service_aspect_extraction_3,ambience_aspect_extraction_3
0,good time ate philly cheese steak,good girl pleasant pay attention service,
1,good also beware happy hour beer,,
2,,hot fresh pricey pretty service,
3,beer hot,,
4,delicious thing see wa beer,,
...,...,...,...
3887,,,
3888,,,
3889,,,
3890,nice sweet coffee wa good cake,,


# Combined everything split train/val/test

In [31]:
dataframe = pd.concat([df, aspect_extraction_1, aspect_extraction_2, aspect_extraction_3], axis = 1)
dataframe.head()

,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,n_photo_user,url_user,restraunt_name,...,other_sentiment,food_aspect_extraction_1,service_aspect_extraction_1,ambience_aspect_extraction_1,food_aspect_extraction_2,service_aspect_extraction_2,ambience_aspect_extraction_2,food_aspect_extraction_3,service_aspect_extraction_3,ambience_aspect_extraction_3
0,ChdDSUhNMG9nS0VJQ0FnSURoN2IzU2hRRRAB,Took my son there for dinner and we had a good...,5 days ago,28:14.0,5,laurent,0,0,https://www.google.com/maps/contrib/1052889272...,Hooters,...,1,philly cheese steak really good,service good girl pleasant pay attention servi...,,,really good service good service service good,,good time ate philly cheese steak,good girl pleasant pay attention service,
1,ChZDSUhNMG9nS0VJQ0FnSURodHVDeWRREAE,Pathetic food. Ordered vegetarian burger. Tast...,a week ago,28:14.0,1,Fa Ris,0,0,https://www.google.com/maps/contrib/1061823533...,Hooters,...,-1,pathetic food order vegetarian burger taste aw...,,,pathetic food burger taste awful awful potato ...,,,good also beware happy hour beer,,
2,ChZDSUhNMG9nS0VJQ0FnSURoLW9HUEhREAE,"Friendly staff, superb service, food was serve...",a week ago,28:14.0,5,Florencio Gali,0,0,https://www.google.com/maps/contrib/1074077044...,Hooters,...,0,food serve hot fresh pricey,friendly staff superb service pretty service s...,place bit outdated might need upgrading,food serve hot,friendly staff pretty service,,,hot fresh pricey pretty service,
3,ChZDSUhNMG9nS0VJQ0FnSURoaXVyTkhREAE,I have yet to find a Hooter's that I didn't li...,2 weeks ago,28:14.0,5,Joshua Manning,0,0,https://www.google.com/maps/contrib/1083143598...,Hooters,...,1,cold beer hot wing,,,cold beer beer hot,,,beer hot,,
4,ChZDSUhNMG9nS0VJQ0FnSURock9MV1pREAE,"Alcohol is expensive everywhere in Singapore, ...",2 weeks ago,28:14.0,4,J.K LEE,0,0,https://www.google.com/maps/contrib/1101703195...,Hooters,...,1,chicken delicious,beer 2 +1 promotion nice take picture staff,,,,,delicious thing see wa beer,,


In [32]:
# shuffling the data
# n = len(dataframe)
# random_index = np.random.RandomState(seed=42).permutation(n) #
# dataframe = dataframe.iloc[random_index]
# dataframe.reset_index(drop=True, inplace=True)
# dataframe.head()

In [33]:
# print(dataframe.shape)
# # 2800 for training
# # 500 for validation
# # rest for testing
# train, val, test = dataframe.iloc[:2800], dataframe.iloc[2800:3300] , dataframe.iloc[3300:]

In [34]:
# train.to_csv("data/train.csv")
# val.to_csv("data/val.csv")
# test.to_csv("data/test.csv")